In [1]:
import os, sys, django, pandas as pd , numpy as np, requests, json, re, itertools, ftfy
from django.db.models import Q
from utils import TJData, setup_env, ElisAPI
# configura notebook para se utilizar recursos do django, se necessário
setup_env() 
from api import models as tj
# objeto para interagir com a minha API
api = ElisAPI('http://0.0.0.0:8000','elisdbadmin','elisdbpassword')

### Comarca
O bloco de código abaixo foi utilizado para analisar e decidir visualmente qual planilha referente a comarcas será utilizada, a planilha mapeada por `comarca` possui a coluna `COD_SERV` possui codigos distintos dos referenciados por demais planilhas
```
com_1 = TJData.read_csv(TJData.csv['comarca'],usecols=['COD_COM','NOME','NOME_REDU','COD_TJ'])
com_2 = TJData.read_csv(TJData.csv['comarca_2'], usecols=['COD_COMA','DESC_COMA','COD_TJ','DESC_REDU'])
com = pd.merge(left=com_1,right=com_2,how='inner',left_on='NOME_REDU', right_on='DESC_REDU')
```

#### Tratamento de dados

### Serventia

``` 
serv_1 = TJData.read_csv(TJData.csv['serventia_2'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'] )
serv_2 = TJData.read_csv(TJData.csv['serventias_full'], usecols=['COD_SERV','DESC_SERV','COD_COMA','DESC_ABRE','DESC_REDU'])
serv = pd.merge(left=serv_1,right=serv_2,how='inner',left_on='COD_SERV', right_on='COD_SERV')
serv_outer = pd.merge(left=serv_1,right=serv_2,how='outer',left_on='COD_SERV', right_on='COD_SERV')
```
#### Tratamento de dados

### Competencias

### Tipo Personagem

### Assunto

### Classe

### Classe assunto

In [100]:
classes_assuntos = TJData.read_csv(TJData.csv['classe_assunto_2'], usecols=['COD_CLASSE','COD_ASSUNTO'])
classes_assuntos = classes_assuntos.rename(columns={'COD_CLASSE':'classe','COD_ASSUNTO':'assunto'})

# # d
# classes = list(map(lambda x : x['cod_classe'], api.response_to_json(api.get('classes'))))
# assuntos = list(map(lambda x : x['cod_assunto'], api.response_to_json(api.get('assuntos'))))
# classes_assuntos = classes_assuntos[(classes_assuntos.classe.isin(classes)) & (classes_assuntos.assunto.isin(assuntos))]
# ca_db = pd.DataFrame(api.response_to_json(api.get('classesassuntos')))
# diff = list(set(zip(classes_assuntos.classe, classes_assuntos.assunto)) - set(zip(ca_db.classe, ca_db.assunto)))
# classes_assuntos = pd.DataFrame(diff, columns=['classe','assunto'])
classes_assuntos = TJData.to_json(classes_assuntos)
classes_assuntos = api.split_list(classes_assuntos,1000)
r = api.concurrent_request(api.post,'classesassuntos',classes_assuntos)

### Tipo movimento

### Tipo Andamento

### Cargo

### Funcionário

### Tipo Decisão Recurso

### Tipo Ato Juiz

### Ato Juiz

### Tipo Documento

### Processo

#### Obtem as serventias com função de Juizado Especial Cível das comarcas de Bangu e Campo Grande

In [2]:
serventias_jec = pd.DataFrame(
    list(map(lambda x: (x.comarca.desc_redu,x.comarca.desc_coma, x.cod_serv, x.desc_serv),
        tj.Serventia.objects.filter(
            Q(desc_serv__contains='Juizado Especial Cível'),
            Q(comarca__desc_redu='BAN') | Q(comarca__desc_redu='CGR')
    ))),
    columns=['comarca.desc_redu','comarca.desc_coma','cod_serv','desc_serv']
)
serventias_jec

,comarca.desc_redu,comarca.desc_coma,cod_serv,desc_serv
0,BAN,Regional de Bangu,204009,Cartório do 17º Juizado Especial Cível
1,BAN,Regional de Bangu,204509,17º Juizado Especial Cível
2,BAN,Regional de Bangu,204025,Cartório do 29º Juizado Especial Cível
3,BAN,Regional de Bangu,204525,29º Juizado Especial Cível
4,CGR,Regional de Campo Grande,205020,Cartório do 26º Juizado Especial Cível
5,CGR,Regional de Campo Grande,205520,26º Juizado Especial Cível
6,CGR,Regional de Campo Grande,205507,18º Juizado Especial Cível
7,CGR,Regional de Campo Grande,205007,Cartório do 18º Juizado Especial Cível


#### Obtem a lista de planilhas para processar resultantes codigos de serventias e do intervalo de anos selecinados 

####  carrega os processos que por algum motivo não tenham sido processados anteriormente

### Andamento Processo (Sentença)

### Personagem

In [5]:
personagem = TJData.read_csv(TJData.csv['personagem'])
personagem_esp = TJData.read_csv(TJData.csv['personagem_especial'])
personagem = TJData.read_csv(TJData.csv['personagem_especial_original'])
personagem = TJData.read_csv(TJData.csv['personagem_processo'])

dict_keys(['assunto', 'assunto_2', 'assunto_3', 'tipo_pedido_mp', 'tipo_vinculo', 'serventias_full', 'ato_juiz', 'ato_juiz_2', 'destinatario', 'tipo_vinculo_processo', 'tipo_andamento', 'classe', 'orgao_mp', 'grupo_processamento', 'tipo_ato_juiz', 'comarca', 'tipo_categoria_serventia', 'tipo_destinatario', 'competencia', 'tipo_remessa', 'rito', 'advogado', 'indice_processo', 'personagem', 'personagem_especial', 'personagem_especial_original', 'personagem_processo', 'cargo', 'classe_assunto_2', 'classe_assunto_primeiro_nivel', 'classe_2', 'comarca_2', 'funcao', 'serventia_2', 'tipo_acao_cnj', 'tipo_acao', 'tipo_andamento_1', 'tipo_andamento_2', 'tipo_andamento_local', 'tipo_categoria_docto_juntada', 'tipo_decisao', 'tipo_decisao_recurso', 'tipo_entidade', 'tipo_movimento', 'tipo_movimento_2', 'tipo_personagem', 'grupo_processamento_2', 'tipo_resultado_audiencia', 'funcionario', 'tipo_documento', 'tipo_documento_juntada', 'tipo_andamento_3', 'processo_unico'])